In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

# Đọc dữ liệu đã gộp
df = pd.read_excel('table_sample/super_sentinel_merge.xlsx')
# Chọn các đặc trưng và nhãn


features = ['brightness', 'br_ratio', 'min_rgb', 'ndi_rb', 'ndvi']
X = df[features]
y = df['Classvalue']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)



In [ ]:
# Train mô hình cơ bản
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Đánh giá
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'class_weight': [None, 'balanced'],
    'bootstrap': [True]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid,
                    scoring='f1',
                    cv=5,
                    n_jobs=-1,
                    verbose=1)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best F1 score:", grid.best_score_)


In [ ]:
best_rf = RandomForestClassifier(
    bootstrap=True,
    class_weight= None,
    max_depth= 10,
    max_features= 'sqrt',
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)
best_rf.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
y_pred = best_rf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))


In [ ]:
import joblib
joblib.dump(best_rf, 'best_random_forest_model.pkl')